In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from pathlib import Path

root_dir = Path("../.data/").resolve()

In [ ]:
df_pkv = pd.read_csv(root_dir / "pkv-entwicklung-vpi.csv", sep=";")
df_pkv["percent"] = df_pkv["Beitrag zur privaten Krankenversicherung"].pct_change()
df_pkv["percent"].iloc[0] = 0
df_pkv.head()

In [ ]:
df_pkv.columns

In [ ]:
df_insurance = pd.read_csv(root_dir / "contribution_rates.csv")
df_insurance.head()

In [ ]:
df_zusatz = pd.read_csv(root_dir / "additional_contribution_rates_with_increase.csv")
df_zusatz.head()

In [ ]:
insurance_increase = df_insurance["Yearly Increase (%)"].values[1:]
insurance_increase

In [ ]:
renteanpassung = pd.read_csv(
    root_dir / "rentenanpassung-verbraucherpreisindex.csv", sep=";"
)
renteanpassung.head()

In [ ]:
zusatz_increase = df_zusatz["Yearly Increase (%)"].values[1:]
zusatz_increase

In [ ]:
zusatz_percentage = [
    x / 100 for x in df_zusatz["Average Additional Contribution Rate (%)"].values
]
zusatz_percentage

In [ ]:
df = pd.read_csv(root_dir / "gkv_contribution_rates.csv")
df["BBG (Month)"] = df["BBG (Month)"] / 100
df["Average GKV Contribution Rate (%)"] = (
    df["Average GKV Contribution Rate (%)"].str.replace(",", ".").astype(float)
)
df.index = df["Year"]
df = df.drop(columns=["Year"])
df = df.sort_index()
df["BBG (Month) pct"] = df["BBG (Month)"].pct_change()
df["BBG (Month) pct"].iloc[0] = 0
df.head()

In [ ]:
# hochstbeitrag_2024 = 843
# age_2024 = 30
# age_rente = 72
# expected_life = 90
# rente_anspruch = 2500

# zusatz_percent_2024 = df_zusatz["Average Additional Contribution Rate (%)"].iloc[-1]
# zusatz_value = 5175 * (zusatz_percent_2024/100)


In [ ]:
max_salary_for_gkv = df["BBG (Month)"].loc[2023]
gkv_percent = df["Average GKV Contribution Rate (%)"].loc[2023] / 100
gkv_basis_cost = max_salary_for_gkv * gkv_percent
zusatz_percent = df_zusatz["Average Additional Contribution Rate (%)"].iloc[-1] / 100
zusatz_cost = max_salary_for_gkv * zusatz_percent

pkv_cost = 300

age_at_start = 30
no_pkv_extra = 60
age_rente = 72
expected_life = 90
rente_anspruch = 2500
kid_pkv_cost = 100
kids = [[30, 48], [32, 50]]

In [ ]:
zusatz_percent

In [ ]:
max_salary_change_bag = df["BBG (Month) pct"].values
gkv_percent_bag = [x / 100 for x in df["Average GKV Contribution Rate (%)"].values]
zusatz_percent_bag = zusatz_percentage
pkv_increase_bag = df_pkv["percent"].values


In [ ]:
# one simulation
salary = max_salary_for_gkv
gkv_p = gkv_percent
zusatz_p = zusatz_percent
rente = rente_anspruch
pkv = pkv_cost
gkv_year_cost = []
pkv_year_cost = []

for year in range(age_at_start, age_rente):
    salary_increase = np.random.choice(max_salary_change_bag)
    salary = salary * (1 + salary_increase)
    gkv_p = np.random.choice(gkv_percent_bag)
    zusatz_p = np.random.choice(zusatz_percent_bag)

    gkv_cost = 12 * (salary * gkv_p + salary * zusatz_p)
    gkv_own_share_cost = gkv_cost / 2
    gkv_year_cost.append(gkv_own_share_cost)

    # print(f"{salary_increase=}, {salary=}, {gkv_p=}, {zusatz_p=}")

    rente_increase = np.random.choice(renteanpassung["Rentenanpassung West"].values)
    rente = rente * (1 + rente_increase / 100)

    pkv_increase = np.random.choice(pkv_increase_bag)
    pkv = pkv * (1 + pkv_increase)

    if year == no_pkv_extra:
        pkv = pkv * 0.9

    pkv_cost = 12 * pkv

    if kids:
        for kid in kids:
            # start and end of care years
            if year > kid[0] and year < kid[1]:
                pkv_cost += kid_pkv_cost * 12

    own_share_cost = pkv_cost / 2
    pkv_year_cost.append(own_share_cost)

    print(f"{year}: GKV: {gkv_cost/12}, PKV: {pkv_cost/12}, Rente: {rente}")

for year in range(age_rente, expected_life):
    rente_increase = np.random.choice(renteanpassung["Rentenanpassung West"].values)
    rente = rente * (1 + rente_increase / 100)

    gkv_p = np.random.choice(gkv_percent_bag)
    zusatz_p = np.random.choice(zusatz_percent_bag)

    gkv_cost = 12 * (rente * gkv_p + rente * zusatz_p)
    gkv_own_share_cost = gkv_cost / 2
    gkv_year_cost.append(gkv_own_share_cost)

    pkv_increase = np.random.choice(pkv_increase_bag)
    pkv = pkv * (1 + pkv_increase)

    pkv_cost = 12 * pkv

    if kids:
        for kid in kids:
            # start and end of care years
            if year > kid[0] and year < kid[1]:
                pkv_cost += kid_pkv_cost * 12

    own_share_cost = pkv_cost - gkv_own_share_cost
    pkv_year_cost.append(own_share_cost)

    print(f"{rente=}, GKV: {gkv_cost/12}, PKV: {pkv_cost/12}")

print(f"GKV: {sum(gkv_year_cost)}, PKV: {sum(pkv_year_cost)}")


In [ ]:
zusatz_percent